In [1]:
from Angrybird import AngryBird
import numpy as np
import matplotlib.pyplot as plt
#import tensorflow as tf
from tqdm.notebook import tqdm

In [2]:
seconds = np.arange(11)
pos = np.zeros((11, 2))
pos[:, 0] = seconds * 10
pos[:, 1] = seconds * 20 - 2 * seconds**2

coeffs = np.load("c.npy")
def main(state):
    t = state[0]
    return coeffs[:2]*pos[int(t)]-coeffs[2:4]*state[1:] + coeffs[4:]

In [3]:
db1=np.load("db1.npy")
db2=np.load("db2.npy")
ub1=np.load("ub1.npy")
ub2=np.load("ub2.npy")
print(ub1 - db1)
print(ub2 - db2)

[ 3.39762732  6.29015667  7.41270714  8.76507905 10.17422219 12.02300831
 14.71947301 15.07288856 17.66715812]
[ 3.67227892  5.79009235  7.35489345  8.64400361  9.6315752  11.989377
 13.32669018 15.0594309  16.80086753]


In [4]:
# Polynome de deg 3
def init_mat():
    fmat = []
    fmat.append(np.zeros((1, 1, 2)))
    for i in range(1, 10):
        fmat.append(np.zeros((i, i, 8)))
    return fmat

def force_from_coeff(x1, x2, coeffs):
    x1 /= 100
    x2 /= 50
    a0, a1, a2, a3, b0, b1, b2, b3 = coeffs
    force1 = a0 + a1*x1 + a2*x1**2 + a3*x1**3
    force2 = b0 + b1*x2 + b2*x2**2 + b3*x2**3
    return np.array([force1, force2])

def force_from_mat(mat, state):
    t, x1, x2 = state
    t = int(t)
    if t == 0:
        return mat[0][0][0]
    else:
        i1 = min(max(int((x1 - db1[t-1]) / (ub1[t-1]-db1[t-1]) * t), 0), t-1)
        i2 = min(max(int((x2 - db2[t-1]) / (ub2[t-1]-db2[t-1]) * t), 0), t-1)
        #return mat[t][i1][i2]
        #print(i1, i2)
        return force_from_coeff(x1, x2, mat[t][i1][i2])

def rand_place():
    i = np.random.randint(0, 2282)
    #print(i)
    if i < 2:
        return 0, 0, 0, i
    else:
        j = 2
        for a in range(1, 10):
            for b in range(a):
                for c in range(a):
                    for d in range(8):
                        if i == j:
                            return a, b, c, d
                        else:
                            j += 1
    print(i)

def rand_places(i):
    places = []
    for _ in range(i):
        places.append(rand_place())
    return places

def tups_update(mat, places, lr, j):
    try:
        for tup in places:
            a, b, c, d = tup
            mat[a][b][c][d] += lr * j
    except:
        print(places)
def use_mat(place_mat, state):
    t, x1, x2 = state
    t = int(t)
    if t == 0:
        place_mat[0][0][0] = 1
    else:
        i1 = min(max(int((x1 - db1[t-1]) / (ub1[t-1]-db1[t-1]) * t), 0), t-1)
        i2 = min(max(int((x2 - db2[t-1]) / (ub2[t-1]-db2[t-1]) * t), 0), t-1)
        #return mat[t][i1][i2]
        #print(i1, i2)
        place_mat[t][i1][i2] = 1

In [5]:
#np.save("vents", vents)
vents = np.load("vents.npy")

In [6]:
#umat = init_mat()
#umat = np.load("umat.npy",allow_pickle=True)
vmat = init_mat()

In [ ]:
env = AngryBird()
restart = True and False
lr = 0.01
m = 1000
if restart:
    step = 1
    all_rewards = []
rewards = np.zeros(m)
average_reward = 300.
while average_reward > 110.:
    place_mat = init_mat()
    if (step+1)%50==0:
        print("Step {}, last cost = {}, mean over 100 rounds = {}".format(step+1, average_reward, np.mean(all_rewards[-100:])))
        np.save("rewards", all_rewards)
    nb=max(10, int(100 / np.sqrt(step)))
    places = rand_places(nb)
    tups_update(vmat, places, lr, 1)
    cost1 = 0.
    #cost2 = 0.
    j_ = 0.
    for z in range(m):
        state = env.reset()
        env.vent=vents[z]
        for i in range(10):
            u_force = force_from_mat(umat, state) + main(state)
            v_force = force_from_mat(vmat, state)
            use_mat(place_mat, state)
            state, _, _, _ = env.step(u_force, v_force)
        env.step(np.zeros(2))
        cost1 += np.sum(env.cost)
        rewards[z] = np.sum(env.cost)
        j_ += env.j_prime_MC()
    for i in range(10):
        vmat[i] *= place_mat[i]
        umat[i] += -j_/m * vmat[i]
    vmat = init_mat()
    step+=1
    average_reward = np.mean(rewards)
    all_rewards.append(average_reward)

Step 50, last cost = 119.25982747843366, mean over 100 rounds = 119.25994748420031
Step 100, last cost = 119.25999953856963, mean over 100 rounds = 119.25995200987929
Step 150, last cost = 119.26035144395395, mean over 100 rounds = 119.26007155013595
Step 200, last cost = 119.26016879073295, mean over 100 rounds = 119.26022155511527
Step 250, last cost = 119.2599486167645, mean over 100 rounds = 119.26018676254111
Step 300, last cost = 119.25965170798737, mean over 100 rounds = 119.25989116229816
Step 350, last cost = 119.25979276201453, mean over 100 rounds = 119.25973030610508
Step 400, last cost = 119.2597668675786, mean over 100 rounds = 119.25979712295765
Step 450, last cost = 119.25976876805981, mean over 100 rounds = 119.2598075160119
Step 500, last cost = 119.25959302267124, mean over 100 rounds = 119.25971661523462
Step 550, last cost = 119.25947843862284, mean over 100 rounds = 119.25960427962269
Step 600, last cost = 119.2597824117811, mean over 100 rounds = 119.259605548017

#### Save
db1, ub1, db2, ub2, umat